In [1]:
import importlib
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [2]:
with open("verdict.txt", "r", encoding= "utf-8") as f: 
    raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5773


In [3]:
enc_sample = enc_text[50:]

Context size determines how many tokens are included in the input

In [4]:
context_size = 4
# The context_size of 4 means that the model is trained to look at a sequence of 4 words (or tokens)
# to predict the next word in the sequence
# The input x is the first 4 tokens [1,2,3,4], and the target y is the next 4 tokens [2,3,4,5]

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [339, 550, 5710, 465]
y:      [550, 5710, 465, 12036]


Processing the inputs along with the targets, which are the inputs shifted by one position, we can then create the next-word prediction tasks as follows: 

In [5]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[339] ----> 550
[339, 550] ----> 5710
[339, 550, 5710] ----> 465
[339, 550, 5710, 465] ----> 12036


In [6]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 he ---->  had
 he had ---->  dropped
 he had dropped ---->  his
 he had dropped his ---->  painting


### Implementing Data Loader

- Step 1: Tokenize the entire text 
- Step 2: Use a sliding window to chunk the book into overlapping sequences of max_length
- Step 3: Return the total number of rows in the dataset
- Step 4: Return a single row from the dataset

In [7]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
    # Tokenize the entire text 
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

    # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range (0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i+1: i+max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
        
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

Now the following code will use GPTDatasetV1 to load the inputs in batches via PyTorch DataLoader: 
- Step1: Initialize the tokenizer
- Step2: Create dataset
- Step3: drop_last = True drops the last batch if it is shorter than the specified batch_size to prevent loss spokes during training
- Step4: The number of CPU processes to use for preprocessing

In [8]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    #Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size = batch_size, 
        shuffle= shuffle, 
        drop_last = drop_last, 
        num_workers= num_workers
    )

    return dataloader

In [9]:
with open("verdict.txt", "r", encoding="utf-8") as f: 
    raw_text = f.read()

In [10]:
import torch 
print("PyTorch version:", torch.__version__)
dataloader = create_dataloader_v1 (raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

PyTorch version: 2.9.0+cpu
[tensor([[10970, 33310,    35, 18379]]), tensor([[33310,    35, 18379,   198]])]


In [11]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[33310,    35, 18379,   198]]), tensor([[   35, 18379,   198, 15749]])]


- Small batch size require less memory during training but lead to noisy model update
- Batch size is a trade-off and hyperparameter to experiment with when training the LLMs

In [12]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[10970, 33310,    35, 18379],
        [  198, 15749, 40417,   198],
        [   40,   550,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   198],
        [  198, 11274,  5891,  1576],
        [  438,   568,   340,   373]])

Targets:
 tensor([[33310,    35, 18379,   198],
        [15749, 40417,   198,    40],
        [  550,  1464,  1807,  3619],
        [  402,   271, 10899,  2138],
        [  257,  7026, 15632,   438],
        [ 2016,   257,   198,   198],
        [11274,  5891,  1576,   438],
        [  568,   340,   373,   645]])


In [20]:
import torch
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [21]:
max_length = 4 
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length,stride=max_length, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [22]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[10970, 33310,    35, 18379],
        [  198, 15749, 40417,   198],
        [   40,   550,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   198],
        [  198, 11274,  5891,  1576],
        [  438,   568,   340,   373]])

Inputs shape:
 torch.Size([8, 4])


Now use the embedding layer to embed these Token IDs into 256-dimensional vectors

In [23]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [24]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [27]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])
